In [1]:
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
 

In [2]:
docs = TextLoader("speech.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
final_docs = text_splitter.split_documents(docs)


In [4]:
embeddings = OllamaEmbeddings(model="gemma:2b")

db = Chroma.from_documents(documents=final_docs,embedding=embeddings)
db

In [5]:
###query

query = "What does the speaker believe is the main reason the United States should enter the war?"
docs = db.similarity_search(query)
docs[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [6]:
##save to local

db = Chroma.from_documents(documents=final_docs,embedding=embeddings, persist_directory="./chroma_db")

In [7]:
#load from local/disk

db2 = Chroma(persist_directory="./chroma_db",embedding_function=embeddings)

In [8]:
docs = db2.similarity_search(query)
docs[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

In [12]:
docs = db2.similarity_search_with_score(query)
docs

[(Document(id='6ce9072e-12cc-484b-91de-44ff247a9fed', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  3531.251953125),
 (Document(id='99ddfb9b-6b83-4db5-a9de-870bd4f6265a', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and

In [13]:
###retriever option

retriever = db.as_retriever()
retriever.invoke(query)

[Document(id='6ce9072e-12cc-484b-91de-44ff247a9fed', metadata={'source': 'speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='99ddfb9b-6b83-4db5-a9de-870bd4f6265a', metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of natio

In [14]:
retriever.invoke(query)[0].page_content

'always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'